# Intel® System Debugger

The System Debug feature of the Intel® System Debugger is a JTAG-based debug solution supporting in-depth debugging and tracing of Intel® Architecture-based System Software and Embedded Applications. It enables developers to debug and trace Intel® Architecture based platforms system-wide, e.g. UEFI / firmware, System-on-Chip peripheral registers, OS kernel and drivers with full OS awareness.

The Intel® System Debugger - System Debug provides Windows* hosted cross-debug solutions for software developers to debug UEFI BIOS, the Linux* kernel sources and dynamically loaded drivers and kernel modules on Intel Atom® processor and Intel® Core™ based devices.

There are two primary ways to establish connection to target- DAL and OpenIPC. We will cover both these approaches in this demo.

## Using DAL 

<img src="PythonCLI01.PNG" alt="PythonCLI-01" style="width:80%;align: left"/>

The Intel® DFx Abstraction Layer (Intel® DAL) provides a rich source of functionality for debugging target platforms and associated silicon. The Intel DAL abstracts away the target-specific details on how to access the features of interest, such as model-specific registers (MSRs), entering and exiting probe mode, various forms of hardware break detects, and more. The Python* Command Line Interface (Python* CLI) provides access to this functionality in an easy-to-use form while still allowing access to the more advanced features of the Intel DAL

In [ ]:
#Change the directory to Intel's DAL from Intel® System Debugger
#Note that the version of DAL installed on the demo machine should match the version given in this cell
%cd "C:/Intel/DAL_1.1831.340.110"

In [ ]:
#Select the right config file based on the target platform
! ConfigConsole.exe

In [ ]:
#Establish target connection; wait for a while and observe green lights on the CCA
! python itpii/startup.py

In [ ]:
# Call the itpii.baseaccess() command to get an instance of the itpii.ItpII class.
import sys, os
dalDir = r"C:/Intel/DAL_1.1831.340.110"
sys.path.insert(0, dalDir)
import itpii
itp = itpii.baseaccess()

In [ ]:
# Check DAL version

itp.cv.version

### Enter/Exit Probe Mode

In [ ]:
# Put all the threads into probe mode

# Output shows a series of messages which describes how each thread entered probe mode and at what address they 
# were executing at the time they were halted
itp.halt()
itp.status()

In [ ]:
# Resume execution of all threads
itp.go()
itp.status()

In [ ]:
# Show current breakall setting
itp.cv.breakall

In [ ]:
# ***********************************************************************************

# Use the control variable called 'breakall' to change the behavior of the Intel DAL in regards to what happens to other threads 
# when a specific thread is told to halt (that is, enter probe mode). 

# If breakall = "off", DAL halts only one specific thread and leaves the other threads running

# If breakall = "global", DAL halts all threads when one specific thread is halted

# Keep any one statement active below, based on the thread halt behavior desired; default value is breakall = "global"

# ***********************************************************************************

# itp.cv.breakall = "off"
itp.cv.breakall = "global"

In [ ]:
# Demonstrate the use of breakall- part 1

# Below statements halt only the first thread, and the other threads are kept running

itp.threads[0].halt()
itp.status()

In [ ]:
# Demonstrate the use of breakall- part 2

# Below statements resume the first thread which was halted in the previous step; the other threads are not affected

itp.threads[0].go()
itp.status()

### View/Modify Registers

In [117]:
# Create aliases for threads
t0 = itp.threads[0]
t1 = itp.threads[1]
t2 = itp.threads[2]
t3 = itp.threads[3]

In [118]:
# View desired registers for debug

# Use "Tab" key to get a list of registers using the below template:
# print t0.state.regs.<tab>

print t0.state.regs.rip
print t0.state.regs.rflags

[64b] 0x0000000089E5D311
[64b] 0x0000000000010206


In [119]:
# Save rflags value

savedRflags = t0.state.regs.rflags.value

In [120]:
# Update rflags value to all 1s (0xfffffffff)

t0.state.regs.rflags = 0xfffffffff
print "Updated rflags value:\n%s"%t0.state.regs.rflags

print "\n\nExpanded view of rflags:"
t0.state.regs.rflags

Updated rflags value:
[64b] 0x0000000FFFFFFFFF


Expanded view of rflags:


rflags(63:0) = 0x0000000FFFFFFFFF
  eflags(31:0) = 0xFFFFFFFF
    flags(15:0) = 0xFFFF
    cf = 0x1
    pf = 0x1
    af = 0x1
    zf = 0x1
    sf = 0x1
    tf = 0x1
    ifl = 0x1
    df = 0x1
    of = 0x1
    iopl(1:0) = 0x3
    nt = 0x1
    rf = 0x1
    vm = 0x1
    ac = 0x1
    vif = 0x1
    vip = 0x1
    id = 0x1

In [121]:
# Restore rflags value

t0.state.regs.rflags = savedRflags
print "Restored flags value is:\n%s"%t0.state.regs.rflags

print "\n\nExpanded view of rflags:"
t0.state.regs.rflags

Restored flags value is:
[64b] 0x0000000000010206


Expanded view of rflags:


rflags(63:0) = 0x0000000000010206
  eflags(31:0) = 0x00010206
    flags(15:0) = 0x0206
    cf = 0x0
    pf = 0x1
    af = 0x0
    zf = 0x0
    sf = 0x0
    tf = 0x0
    ifl = 0x1
    df = 0x0
    of = 0x0
    iopl(1:0) = 0x0
    nt = 0x0
    rf = 0x1
    vm = 0x0
    ac = 0x0
    vif = 0x0
    vip = 0x0
    id = 0x0

### Check/Modify Target Status

In [122]:
# Check target power status

itp.cv.targpower

True

In [123]:
# Check processor mode

t0.cv.asmmode

use16

In [124]:
# List the known TAP devices; the devices primarily hold information about the silicon or portion of silicon it is representing

itp.devicelist

 DID  DP TP  SC Alias                Type                Step Idcode     BusType  P/D/ C/T  Enabled
----------------------------------------------------------------------------------------------------
   0  0   0  1  WHL_U_UC0            WHL_U_UC            W0   0xBA76D013 JTAG     0/0/ -/-  Yes
   1  0   0  1  WHL_CBO0             WHL_CBO             W0              JTAG     0/0/ -/-  Yes
   2  0   0  1  WHL_CBO1             WHL_CBO             W0              JTAG     0/0/ -/-  Yes
   3  0   0  1  WHL_CBO2             WHL_CBO             W0              JTAG     0/0/ -/-  Yes
   4  0   0  1  WHL_CBO3             WHL_CBO             W0              JTAG     0/0/ -/-  Yes
   5  0   0  1  P0                   WHL                 W0              JTAG     0/0/ 0/0  Yes
   6  0   0  1  P1                   WHL                 W0              JTAG     0/0/ 0/1  Yes
   7  0   0  1  P2                   WHL                 W0              JTAG     0/0/ 1/0  Yes
   8  0   0  1  P3             

In [125]:
# Display processor's running state and execution mode

itp.status()

Status for       : WHL_C0_T0
  Processor      : Halted
  Processor mode : Protected, Paged, SixtyFourBit
Status for       : WHL_C0_T1
  Processor      : Halted
  Processor mode : Protected, Paged, SixtyFourBit
Status for       : WHL_C1_T0
  Processor      : Halted
  Processor mode : Protected, Paged, SixtyFourBit
Status for       : WHL_C1_T1
  Processor      : Halted
  Processor mode : Protected, Paged, SixtyFourBit
Status for       : WHL_C2_T0
  Processor      : Halted
  Processor mode : Protected, Paged, SixtyFourBit
Status for       : WHL_C2_T1
  Processor      : Halted
  Processor mode : Protected, Paged, SixtyFourBit
Status for       : WHL_C3_T0
  Processor      : Halted
  Processor mode : Protected, Paged, SixtyFourBit
Status for       : WHL_C3_T1
  Processor      : Halted
  Processor mode : Protected, Paged, SixtyFourBit



In [134]:
# Display the cause and address of the recent break event for the selected threads specified by the variable tid

tid = 2

print "Status of recent break event for thread[%d]:"%tid
itp.threads[tid].cv.cause



Status of recent break event for thread[2]:


MWAIT C1 break at 0x38:0000000089E5DDF7

In [135]:
# Perform disassembly for the n instructions from the current instruction pointer for the selected threads 
# specified by the variable tid

tid = 0
n   = 10
itp.threads[tid].asm("$", n)

0x38:0000000089E5D311  c3                                ret
0x38:0000000089E5D312  cc                                int3
0x38:0000000089E5D313  cc                                int3
0x38:0000000089E5D314  cc                                int3
0x38:0000000089E5D315  cc                                int3
0x38:0000000089E5D316  cc                                int3
0x38:0000000089E5D317  cc                                int3
0x38:0000000089E5D318  cc                                int3
0x38:0000000089E5D319  cc                                int3
0x38:0000000089E5D31A  cc                                int3



In [136]:
# Dump memory contents as specifed below:
# tid                --> selected thread
# start_addr         --> first address from where to start dump
# no_of_words        --> number of words to be read in the dump file
# wordsize_in_bytes  --> 1 for 8 bit word
#                        2 for 16 bit word
#                        4 for 32 bit word 
#                        8 for 64 bit word 

tid = 0
start_addr = "0x89E5DE00P"
no_of_words = 100
wordsize_in_bytes = 8

itp.threads[tid].memdump(start_addr, no_of_words, wordsize_in_bytes)

0x0000000089E5DE00P: ccc3c88948f1230f cccccccccccccccc 
0x0000000089E5DE10P: ccccccccc3c8210f cccccccccccccccc 
0x0000000089E5DE20P: ccccccccc3d8210f cccccccccccccccc 
0x0000000089E5DE30P: ccccccccc309010f cccccccccccccccc 
0x0000000089E5DE40P: ccc39d19010ffa9c cccccccccccccccc 
0x0000000089E5DE50P: ccccccccc301010f cccccccccccccccc 
0x0000000089E5DE60P: 83485340ccc3589c 8b4860428b4820ec 
0x0000000089E5DE70P: 00008378058948d9 e800008369158948 
0x0000000089E5DE80P: a50d8d48000016e0 00001704e8000075 
0x0000000089E5DE90P: 4800008369158d48 61e8000075720d8d 
0x0000000089E5DEA0P: 49e8cb8b48000016 5b20c48348000008 
0x0000000089E5DEB0P: 28ec8348ccccccc3 f4dee80975c98545 
0x0000000089E5DEC0P: 834117ebc033ffff fff4dfe8077502f9 
0x0000000089E5DED0P: 000003b848efebff c483488000000000 
0x0000000089E5DEE0P: 28ec8348ccccc328 c6c033fffff4a3e8 
0x0000000089E5DEF0P: 834801000082da05 28ec8348ccc328c4 
0x0000000089E5DF00P: c6c033fffff4fbe8 834800000082c205 
0x0000000089E5DF10P: 75d28548ccc328c4 0000000002

### Handling Breakpoints

In [137]:
# Set a breakpoint

itp.brnew("0x100P", "io")

In [138]:
# View breakpoints

itp.brget()

[WHL_C0_T0] br["#001"] = 0x0000000000000100P io global dbreg = 0, enabled = True
[WHL_C0_T0] br["#009"] = 0x0000000000000100P io global dbreg = 1, enabled = True
[WHL_C0_T0] br["#017"] = 0x0000000000000100P io global dbreg = 2, enabled = True
[WHL_C0_T1] br["#002"] = 0x0000000000000100P io global dbreg = 0, enabled = True
[WHL_C0_T1] br["#010"] = 0x0000000000000100P io global dbreg = 1, enabled = True
[WHL_C0_T1] br["#018"] = 0x0000000000000100P io global dbreg = 2, enabled = True
[WHL_C1_T0] br["#003"] = 0x0000000000000100P io global dbreg = 0, enabled = True
[WHL_C1_T0] br["#011"] = 0x0000000000000100P io global dbreg = 1, enabled = True
[WHL_C1_T0] br["#019"] = 0x0000000000000100P io global dbreg = 2, enabled = True
[WHL_C1_T1] br["#004"] = 0x0000000000000100P io global dbreg = 0, enabled = True
[WHL_C1_T1] br["#012"] = 0x0000000000000100P io global dbreg = 1, enabled = True
[WHL_C1_T1] br["#020"] = 0x0000000000000100P io global dbreg = 2, enabled = True
[WHL_C2_T0] br["#005"] = 0x0

### Usage of Debug Scripts

In [139]:
# Demonstrates the use of writing a small script to perform a specific debugging task

# Function resetpower() powercycles the target, puts all the cores in probe mode and 
# displays the status of all the halted cores

def resetpower():     
    import os  
    itp.sleep(3)
    itp.resettarget()
    if t0.cv.isrunning:
        itp.halt()
    itp.status()

resetpower()

	Processor(s) found in halted state, releasing them before reset target
	Note:	Configuration on all debug ports has started
	Target Configuration: WHL_CNP_OpenDCI_CCA_[DbC]_ReferenceSettings
	Note:	Target reset has occurred
	Note:	Configuration on all debug ports has finished
	[WHL_C0_T0]  Halt Command break at 0x38:0000000089E9B3A6
	[WHL_C0_T1]  MWAIT C1 break at 0x38:0000000089E5DDF7
	[WHL_C1_T0]  MWAIT C1 break at 0x38:0000000089E5DDF7
	[WHL_C1_T1]  MWAIT C1 break at 0x38:0000000089E5DDF7
	[WHL_C2_T0]  MWAIT C1 break at 0x38:0000000089E5DDF7
	[WHL_C2_T1]  MWAIT C1 break at 0x38:0000000089E5DDF7
	[WHL_C3_T0]  MWAIT C1 break at 0x38:0000000089E5DDF7
	[WHL_C3_T1]  MWAIT C1 break at 0x38:0000000089E5DDF7
Status for       : WHL_C0_T0
  Processor      : Halted
  Processor mode : Protected, Paged, SixtyFourBit
Status for       : WHL_C0_T1
  Processor      : Halted
  Processor mode : Protected, Paged, SixtyFourBit
Status for       : WHL_C1_T0
  Processor      : Halted
  Processor mode : Pro

## Using OpenIPC

In [ ]:
! python -V

In [ ]:
import ipccli
itp = ipccli.baseaccess()

In [ ]:
itp.cv.version

### Enter/Exit Probe Mode

In [ ]:
# Put all the threads into probe mode

# Output shows a series of messages which describes how each thread entered probe mode and at what address they 
# were executing at the time they were halted
itp.halt()
itp.status()

In [ ]:
# Resume execution of all threads
itp.go()
itp.status()

### View/Modify Registers

In [ ]:
# Create aliases for threads
t0 = itp.threads[0]
t1 = itp.threads[1]

In [ ]:
# View desired registers for debug

# Use "Tab" key to get a list of registers using the below template:
# print t0.state.regs.<tab>

print t0.state.regs.ebx
print t0.state.regs.rflags
print t0.state.regs.xmm0

In [ ]:
savedRflags = t0.state.regs.rflags
savedRflags

In [ ]:
# Update rflags value to all 1s (0xfffffffff)

t0.state.regs.rflags = 0xfffffffff
print "Updated rflags value:\n%s"%t0.state.regs.rflags

In [ ]:
# Restore rflags value

t0.state.regs.rflags = savedRflags
print "Restored flags value is:\n%s"%t0.state.regs.rflags

In [ ]:
# Check MSR (Model Specific Register) values

itp.threads.msr(0x200)

### Check/Modify Target Status

In [ ]:
# Check target power status

itp.cv.targpower

In [ ]:
# Check processor mode for the selected threads specified by the variable 'tid'

tid = 0
itp.threads[tid].cv.asmmode

In [ ]:
# List the known TAP devices; the devices primarily hold information about the silicon or portion of silicon it is representing

itp.devicelist

In [ ]:
# Display processor's running state and execution mode
itp.threads.thread_status()

In [ ]:
# Display the address of the last known break event for the selected threads specified by the variable tid

tid = 0
print itp.threads[tid].break_address()

In [ ]:
# Dump memory contents as specifed below:
# tid                --> selected thread
# start_addr         --> first address from where to start dump
# no_of_words        --> number of words to be read in the dump file
# wordsize_in_bytes  --> 1 for 8 bit word
#                        2 for 16 bit word
#                        4 for 32 bit word 
#                        8 for 64 bit word 

tid = 0
start_addr = "0x88EB5CE3P"
no_of_words = 100
wordsize_in_bytes = 2

itp.threads[tid].memdump(start_addr, no_of_words, wordsize_in_bytes)

### Handling Breakpoints

In [ ]:
# Set a breakpoint

itp.brnew("0x100P", "io")

In [ ]:
# View breakpoints

itp.brget()

### Basic Bitwise Operations

In [ ]:
reg1 = itp.BitData(8, 0xab)
reg2 = itp.BitData(8, 0xcd)
print "reg1 in hex = " +reg1.ToHex()
print "reg2 in hex = " +reg2.ToHex()

# Append reg2 to MSB of reg1

reg3 = reg1.Append(reg2)

# Display reg3 in hex and binary
print "reg3 in hex = " +reg3.ToHex()
print "reg3 in binary = " +reg3.ToBinary()

# Count number of 1s in a byte/word
total_one_bits = reg3.PopCount()
print "total_one_bits in reg3= %d"%total_one_bits


## Launch System Debugger GUI

In [ ]:
%%capture
! C:\IntelSWTools\system_studio_2018_nda\iss_ide_eclipse-launcher.bat